# Neural Networks Course - LEONARDO H. DE MORAES - Final Project  

**Name:** Leonardo H. de Moraes  
**Student ID:** 1711609  

**Course:** Neural Networks - Polytechnic Institute of Guarda (IPG)  
**Acronym:** LEI  
**Department:** School of Technology and Management  
**Degree:** Bachelor's  
**Program:** Computer Engineering  
**Academic Year:** 2024/2025  
**Professor:** Noel de Jesus Mendonça Lopes  

---

## Project Objective  
The goal of this project is to develop predictive models for the exchange rates of the US Dollar (**USD**) and the Japanese Yen (**JPY**) against the Euro (**EUR**). Students will use neural networks to build and compare different models.  

In [1]:
## == Import Libs
import pandas as pd
import numpy as np
import requests
import dotenv
import os
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
from dataset_utils.extract_data import *
## == Import libs 


## == Set global variables
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = '2015-01-01'

In [2]:
## == Load globals variables
dotenv.load_dotenv(dotenv.find_dotenv())

ALPHA_VANTAGE_API_KEY = os.getenv('ALPHA_VANTAGE_API_KEY')
FRED_API_KEY = os.getenv('FRED_API_KEY')
EIA_OIL_API_KEY = os.getenv('EIA_OIL_API_KEY')


In [3]:
## == Extract data from ALPHA_VINTAGE_API_KEY
df_usd_eur = get_exchange_rate_data(start_date, end_date, 'USD', 'EUR', ALPHA_VANTAGE_API_KEY)
df_jpy_eur = get_exchange_rate_data(start_date, end_date, 'JPY', 'EUR', ALPHA_VANTAGE_API_KEY)


Obtendo dados de câmbio USD/EUR de 2015-01-01 a 2025-05-24...
Obtendo dados de câmbio JPY/EUR de 2015-01-01 a 2025-05-24...


In [4]:
## == Extract data from EIA_OIL_API_KEY
df_oil = get_oil_prices(start_date, end_date, EIA_OIL_API_KEY)
df_oil

Obtendo dados de preço do petróleo de 2015-01-01 a 2025-05-24...


,date,oil_price
0,2015-01-02,1.704
1,2015-01-02,1.589
2,2015-01-02,1.492
3,2015-01-02,1.294
4,2015-01-02,1.559
...,...,...
4995,2016-10-13,1.498
4996,2016-10-13,1.563
4997,2016-10-13,0.583
4998,2016-10-13,50.470


In [ ]:
## == Extract data from FRED API
interest_df = get_interest_rates(start_date, end_date, FRED_API_KEY)
interest_df

Obtendo dados de taxas de juros de 2015-01-01 a 2025-05-24...


,date,interest_rate
0,2015-01-01,0.06
1,2015-01-02,0.12
2,2015-01-03,0.12
3,2015-01-04,0.12
4,2015-01-05,0.12
...,...,...
3790,2025-05-18,4.33
3791,2025-05-19,4.33
3792,2025-05-20,4.33
3793,2025-05-21,4.33


In [ ]:

# Função para obter dados do índice S&P 500
def get_sp500(start_date, end_date):
    print(f"Obtendo dados do S&P 500 de {start_date} a {end_date}...")
    
    # Usando a API Alpha Vantage (substitua pela sua chave)
    api_key = 'HG4ZKK1CE0LTWLAR'  # Obtenha uma em: https://www.alphavantage.co/
    
    if api_key == 'YOUR_ALPHAVANTAGE_API_KEY':
        print("AVISO: Usando dados simulados para S&P 500 (sem chave de API)")
        # Criar dados simulados para demonstração
        date_range = pd.date_range(start=start_date, end=end_date, freq='D')
        sp500 = np.random.uniform(2000, 5000, len(date_range))  # Valores simulados entre 2000 e 5000
        return pd.DataFrame({'date': date_range, 'sp500': sp500})
    
    try:
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=SPY&outputsize=full&apikey={api_key}&datatype=json"
        response = requests.get(url)
        data = response.json()
        
        if 'Time Series (Daily)' not in data:
            print("Erro ao obter dados do S&P 500:", data.get('Note', 'Unknown error'))
            return pd.DataFrame()
        
        dates = []
        closes = []
        for date, values in data['Time Series (Daily)'].items():
            if date >= start_date and date <= end_date:
                dates.append(date)
                closes.append(float(values['4. close']))
        
        df = pd.DataFrame({
            'date': pd.to_datetime(dates),
            'sp500': closes
        })
        
        return df.sort_values('date').reset_index(drop=True)
    
    except Exception as e:
        print(f"Erro ao acessar API Alpha Vantage: {e}")
        return pd.DataFrame()


In [ ]:
sp500_df = get_sp500(start_date, end_date)
sp500_df

In [ ]:
# Função principal para construir o dataframe completo
def build_complete_dataset(start_date, end_date):
    # Obter dados de câmbio para ambos os pares
    usd_eur_df = get_exchange_rate_data(start_date, end_date, 'USD', 'EUR')
    jpy_eur_df = get_exchange_rate_data(start_date, end_date, 'JPY', 'EUR')
    
    # Combinar os dados de câmbio primeiro
    df = usd_eur_df.merge(jpy_eur_df, on='date', how='outer')
    
    # Obter outros dados econômicos
    oil_df = get_oil_prices(start_date, end_date)
    interest_df = get_interest_rates(start_date, end_date)
    sp500_df = get_sp500(start_date, end_date)

    
    # Juntar todos os dataframes
    for data in [oil_df, interest_df, sp500_df]:
        if not data.empty:
            df = df.merge(data, on='date', how='left')
    
    # Preencher valores ausentes (substituição do fillna obsoleto)
    df = df.ffill()  # Preenche para frente
    df = df.bfill()  # Preenche para trás (caso ainda haja NAs no início)
    
    # Adicionar features temporais
    df['day_of_week'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    
    # Adicionar diferença percentual diária para todas as taxas de câmbio
    for col in ['exchange_rate_USD_EUR', 'exchange_rate_JPY_EUR', 'oil_price', 'sp500']:
        if col in df.columns:
            df[f'{col}_pct_change'] = df[col].pct_change() * 100
    
    # Adicionar médias móveis
    windows = [7, 30, 90]
    for window in windows:
        for pair in ['USD_EUR', 'JPY_EUR']:
            col = f'exchange_rate_{pair}'
            if col in df.columns:
                df[f'{col}_ma_{window}'] = df[col].rolling(window=window).mean()
        if 'oil_price' in df.columns:
            df[f'oil_price_ma_{window}'] = df['oil_price'].rolling(window=window).mean()
    
    # Remover linhas com valores NaN
    df.dropna(inplace=True)
    
    return df

In [ ]:
# Construir o dataset completo
print("\nIniciando construção do dataset...")
final_df = build_complete_dataset(start_date, end_date)
final_df

In [ ]:
# Verificar o resultado
print("\nDataset construído com sucesso!")
print(f"Colunas disponíveis: {final_df.columns.tolist()}")
print(f"Número de observações: {len(final_df)}")


In [ ]:
# Salvar o dataset
output_file = f"../data/dateset/multi_forex_dataset_{start_date}_to_{end_date}.csv"
final_df.to_csv(output_file, index=False)
print(f"\nDataset salvo como {output_file}")

In [ ]:
# Plotar visualizações
plt.figure(figsize=(15, 10))

# Taxas de câmbio
plt.subplot(2, 2, 1)
plt.plot(final_df['date'], final_df['exchange_rate_USD_EUR'], label='USD/EUR')
plt.plot(final_df['date'], final_df['exchange_rate_JPY_EUR']*100, label='JPY/EUR (x100)')
plt.title('Taxas de Câmbio')
plt.xlabel('Date')
plt.ylabel('Rate')
plt.legend()

# Preço do petróleo
plt.subplot(2, 2, 2)
plt.plot(final_df['date'], final_df['oil_price'])
plt.title('Oil Price (Brent)')
plt.xlabel('Date')
plt.ylabel('USD per Barrel')

# S&P 500
plt.subplot(2, 2, 3)
plt.plot(final_df['date'], final_df['sp500'])
plt.title('S&P 500 Index')
plt.xlabel('Date')
plt.ylabel('Index Value')

# Taxa de juros
plt.subplot(2, 2, 4)
plt.plot(final_df['date'], final_df['interest_rate'])
plt.title('Interest Rate (FED Funds)')
plt.xlabel('Date')
plt.ylabel('Rate (%)')

plt.tight_layout()
plt.show()